In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D


In [ ]:
# Step 2: Download stock data
stock_data = yf.download('AAPL', start='2020-01-01', end='2024-01-01')
prices = stock_data['Close'].values.reshape(-1, 1)

In [ ]:
# Step 3: Preprocessing - Scaling data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_prices, seq_length)


In [ ]:
# Step 4: Build Transformer Model in TensorFlow
def build_transformer_model(seq_length, input_dim, embed_dim=64, num_heads=4, ff_dim=128, dropout_rate=0.1):
    inputs = Input(shape=(seq_length, input_dim))

    # Linear projection (embedding)
    x = Dense(embed_dim)(inputs)

    # Multi-Head Attention Layer
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
    attn_output = Dropout(dropout_rate)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Feed Forward Network
    ff_output = Dense(ff_dim, activation='relu')(out1)
    ff_output = Dense(embed_dim)(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ff_output)

    # Global Pooling & Output
    pooled = GlobalAveragePooling1D()(out2)
    outputs = Dense(1)(pooled)

    model = tf.keras.Model(inputs, outputs)
    return model

model = build_transformer_model(seq_length, input_dim=1)
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# Step 5: Train the model
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

In [ ]:
# Step 6: Predict & Visualize
predicted = model.predict(X)
predicted_prices = scaler.inverse_transform(predicted)
actual_prices = scaler.inverse_transform(y)

plt.figure(figsize=(12, 6))
plt.plot(actual_prices, label='Actual Price', color='blue')
plt.plot(predicted_prices, label='Predicted Price', color='orange')
plt.legend()
plt.title('Stock Price Prediction using Transformer (TensorFlow)')
plt.show()

In [1]:
import yfinance as yf

nifty = yf.Ticker("^NSEI")
df = nifty.history(period="5d", interval="30m")

                           Volume
Datetime                         
2025-02-24 09:30:00+05:30       0
2025-02-24 10:00:00+05:30       0
2025-02-24 10:30:00+05:30       0
2025-02-24 11:00:00+05:30       0
2025-02-24 11:30:00+05:30       0
2025-02-24 12:00:00+05:30       0
2025-02-24 12:30:00+05:30       0
2025-02-24 13:00:00+05:30       0
2025-02-24 13:30:00+05:30       0
2025-02-24 14:00:00+05:30       0
2025-02-24 14:30:00+05:30       0
2025-02-24 15:00:00+05:30       0
2025-02-25 09:30:00+05:30       0
2025-02-25 10:00:00+05:30       0
2025-02-25 10:30:00+05:30       0
2025-02-25 11:00:00+05:30       0
2025-02-25 11:30:00+05:30       0
2025-02-25 12:00:00+05:30       0
2025-02-25 12:30:00+05:30       0
2025-02-25 13:00:00+05:30       0
2025-02-25 13:30:00+05:30       0
2025-02-25 14:00:00+05:30       0
2025-02-25 14:30:00+05:30       0
2025-02-25 15:00:00+05:30       0
2025-02-27 09:30:00+05:30       0
2025-02-27 10:00:00+05:30       0
2025-02-27 10:30:00+05:30       0
2025-02-27 11:

In [2]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2025-02-24 09:30:00+05:30,22621.400391,22644.349609,22564.050781,22581.650391,0,0.0,0.0
2025-02-24 10:00:00+05:30,22582.550781,22590.050781,22548.750000,22568.050781,0,0.0,0.0
2025-02-24 10:30:00+05:30,22567.349609,22579.349609,22552.000000,22553.800781,0,0.0,0.0
2025-02-24 11:00:00+05:30,22555.349609,22584.599609,22550.099609,22574.250000,0,0.0,0.0
2025-02-24 11:30:00+05:30,22574.300781,22620.050781,22568.650391,22590.050781,0,0.0,0.0
2025-02-24 12:00:00+05:30,22589.500000,22595.300781,22558.599609,22562.599609,0,0.0,0.0
2025-02-24 12:30:00+05:30,22561.949219,22587.449219,22553.599609,22577.099609,0,0.0,0.0
2025-02-24 13:00:00+05:30,22576.949219,22591.000000,22546.199219,22560.500000,0,0.0,0.0
2025-02-24 13:30:00+05:30,22560.199219,22565.250000,22520.550781,22546.199219,0,0.0,0.0


In [5]:
# Reliance ticker for NSE
reliance = yf.Ticker("TSLA")

# Fetch historical data (last 5 days, 30-minute interval)
df = reliance.history(period="5d", interval="30m")

# Display the data
print(df)

                                 Open        High         Low       Close  \
Datetime                                                                    
2025-02-25 09:30:00-05:00  327.130005  327.450012  312.500000  313.100006   
2025-02-25 10:00:00-05:00  313.200012  313.200012  305.390015  306.369995   
2025-02-25 10:30:00-05:00  306.440002  308.760010  301.410004  304.169495   
2025-02-25 11:00:00-05:00  304.350006  304.739502  300.109985  301.309998   
2025-02-25 11:30:00-05:00  301.339996  306.465698  298.779999  305.885895   
2025-02-25 12:00:00-05:00  305.950012  305.950012  301.830811  303.420013   
2025-02-25 12:30:00-05:00  303.494995  305.190002  301.720001  303.640106   
2025-02-25 13:00:00-05:00  303.640015  306.369995  301.959991  302.369995   
2025-02-25 13:30:00-05:00  302.399994  304.298004  300.660004  300.785004   
2025-02-25 14:00:00-05:00  300.785004  301.549988  297.251190  298.304993   
2025-02-25 14:30:00-05:00  298.304993  302.149994  298.160095  301.470093   

In [7]:
import yfinance as yf

# Apple stock (AAPL)
aapl = yf.Ticker("AAPL")

# Fetch past 5 days, 30-minute interval data
df = aapl.history(period="5d", interval="30m")

df


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2025-02-25 09:30:00-05:00,248.000000,249.520004,247.100006,247.824997,8004180,0.0,0.0
2025-02-25 10:00:00-05:00,247.824997,247.899994,244.910004,245.110001,4539234,0.0,0.0
2025-02-25 10:30:00-05:00,245.130005,246.360001,244.910004,246.074997,2869444,0.0,0.0
2025-02-25 11:00:00-05:00,246.089996,246.869995,245.649994,246.524994,2043292,0.0,0.0
2025-02-25 11:30:00-05:00,246.535004,249.979996,246.430099,249.542007,4223755,0.0,0.0
2025-02-25 12:00:00-05:00,249.565002,249.690002,248.880005,249.580002,2487943,0.0,0.0
2025-02-25 12:30:00-05:00,249.580002,249.619995,248.080002,248.169998,1896313,0.0,0.0
2025-02-25 13:00:00-05:00,248.179993,248.839996,247.729996,247.979996,1295371,0.0,0.0
2025-02-25 13:30:00-05:00,247.970001,248.619995,247.789993,248.139999,1446366,0.0,0.0
